In [1]:
import findspark
from pyspark import SparkContext

In [2]:
findspark.init()

In [3]:
sc = SparkContext(master="local", appName="TransformacionesyAcciones")

### paralelize

In [4]:
rdd1 = sc.parallelize([1,2,3])

In [6]:
rdd1.collect()

[1, 2, 3]

In [7]:
rdd1.take(2)

[1, 2]

### Clase 2 : map, take, count, group, filter

In [5]:
equipos_olimpicos = sc.textFile("C:\\Users\\juan-\\Downloads\\files\\paises.csv") \
    .map(lambda line : line.split(","))

In [9]:
equipos_olimpicos.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

In [10]:
sc.stop()

In [6]:
equipos_olimpicos.map(lambda x: (x[2])).distinct().count()

231

In [7]:
equipos_olimpicos.map(lambda x : (x[2],x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [ ]:
equipos_olimpicos.map(lambda x : (x[2],x[1])).groupByKey()\
    .mapValues(list).take(5)

In [ ]:
equiposargentos = equipos_olimpicos.filter(lambda l : "ARG" in l)
equiposargentos.collect()

In [10]:
equipos_olimpicos.count()

1185

In [12]:
#no me funcionó
equipos_olimpicos.countApprox(10)

1185

### clase 3: union, top, join, take sample

#### carga de archivos

In [16]:
files_path = "C:\\Users\\juan-\\Downloads\\files\\"

In [31]:
deportistaOlimpico = sc.textFile(files_path+"deportista.csv") \
    .map(lambda l : l.split(',')) 

In [32]:
deportistaOlimpico2 = sc.textFile(files_path+"deportista2.csv") \
    .map(lambda l : l.split(',')) 

In [34]:
deportistaOlimpico = deportistaOlimpico.union(deportistaOlimpico2)

In [36]:
deportistaOlimpico.count()

135572

In [38]:
deportistaOlimpico.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [39]:
equipos_olimpicos.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [41]:
deportistaOlimpico.map(lambda l : [l[-1],l[:-1]]).take(1)

[['equipo_id',
  ['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso']]]

In [42]:
equipos_olimpicos.map(lambda x : [x[0],x[2]]).take(1)

[['id', 'sigla']]

#### join

In [40]:
deportistaOlimpico.map(lambda l : [l[-1],l[:-1]]) \
    .join(equipos_olimpicos.map(lambda x : [x[0],x[2]])) \
    .take(6)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN')),
 ('199', (['3605', 'An Weijiang', '1', '22', '178', '72'], 'CHN'))]

In [43]:
deportistaOlimpico.map(lambda l : [l[-1],l[:-1]]) \
    .join(equipos_olimpicos.map(lambda x : [x[0],x[2]])) \
    .takeSample(False, 6, 25)

[('362', (['131505', 'Steven Woodburn', '1', '24', '185', '90'], 'FRA')),
 ('967', (['13626', 'Jill Brresen', '2', '22', '170', '57'], 'RSA')),
 ('482', (['44299', 'Gumundur Gumundsson', '1', '23', '174', '77'], 'ISL')),
 ('970', (['68062', 'Lee MinHui', '2', '28', '174', '65'], 'KOR')),
 ('794', (['92442', 'Luis Paz Zoldan', '1', '19', '187', '82'], 'PER')),
 ('413', (['26822', 'Jared Mark Deacon', '1', '24', '185', '77'], 'GBR'))]

#### resultados.csv

In [46]:
resultado = sc.textFile(files_path+"resultados.csv") \
    .map(lambda l: l.split(','))

In [47]:
resultado.take(3)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['1', 'NA', '1', '39', '1'],
 ['2', 'NA', '2', '49', '2']]

In [49]:
resultado.filter(lambda l : 'NA' not in l[1]).take(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4']]

In [50]:
deportistas_paises = deportistaOlimpico.map(lambda l : [l[-1],l[:-1]]) \
    .join(equipos_olimpicos.map(lambda x : [x[0],x[2]]) )

In [51]:
deportistas_paises.take(3)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN'))]

#### operaciones númericas